In [1]:
import pandas as pd
import numpy as np
import datetime
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
mimic_lab_MV = pd.read_csv('./data/mimic_labs_TS_metavision.csv')
mimic_lab_CV = pd.read_csv('./data/mimic_labs_TS_carevue.csv')

In [3]:
mimic_lab_MV.head()

,row_id,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,flag,expire_flag,dbsource
0,11021,23,124321,50868,2157-10-21 13:40:00,15,15.0,mEq/L,NaN,0,metavision
1,11022,23,124321,50882,2157-10-21 13:40:00,25,25.0,mEq/L,NaN,0,metavision
2,11023,23,124321,50893,2157-10-21 13:40:00,7.8,7.8,mg/dL,abnormal,0,metavision
3,11024,23,124321,50902,2157-10-21 13:40:00,108,108.0,mEq/L,NaN,0,metavision
4,11025,23,124321,50912,2157-10-21 13:40:00,0.6,0.6,mg/dL,NaN,0,metavision


## **get reasonable limit to forward or backward fill (apply "quantile(0.8)" on all patients' time-until-remeasure values for each itemid)**

* **First get common temporal variables between 2 eras for lab data**

In [4]:
temporal_vars_lab_mv = set(mimic_lab_MV["itemid"].unique())
temporal_vars_lab_cv = set(mimic_lab_CV["itemid"].unique())
common_vars_mv_cv_lab = temporal_vars_lab_mv.intersection(temporal_vars_lab_cv)
print(len(temporal_vars_lab_cv))
print(len(temporal_vars_lab_mv))
print(len(common_vars_mv_cv_lab)) # only need to deal with 530 common temporal variables between two eras


581
576
530


* **Secondly get those common itemids' value_counts(), delete those itemid with only count 1, which represents there is no way to compute the remeasure time. After this operation, the number of itemid turns from 530 to 500**

In [5]:
common_itemid_distribution = mimic_lab_MV[mimic_lab_MV["itemid"].isin(common_vars_mv_cv_lab)]["itemid"].value_counts().rename_axis('common_itemid_lab_cv_mv').reset_index(name='counts')
print(common_itemid_distribution.shape)
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(common_itemid_distribution)
final_common_itemid = common_itemid_distribution[common_itemid_distribution["counts"] > 1]
print(final_common_itemid.shape)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(final_common_itemid)

    

(530, 2)
(500, 2)


,common_itemid_lab_cv_mv,counts
0,51221,48832
1,50820,45375
2,50971,42874
3,50902,42367
4,50821,42272
5,50804,42270
6,50802,42268
7,50818,42265
8,50983,41851
9,50912,40392


* **Then get reasonable limit to forward or backward fill:**
    * 1) group by itemid in "final_common_itemid", get group DataFrame: "cur_itemid_group"
    * 2) then groupby hadm_id in each group after grouping by "itemid": "patient_group_for_cur_itemid"
    * 3) compute all the patients' all recordings' intervals on current itemid, storage the time-until-remeasure values for current itemid in a list, and then compute the final fill limit for current itemid in DataFrame "itemid_fill_limit"
    * DataFrame "itemid_fill_limit" is a DataFrame with shape (1, itemid_num)


In [6]:
final_itemid_num = final_common_itemid.shape[0]

itemid_fill_limit = pd.DataFrame(np.zeros((1, final_itemid_num)), columns=list(final_common_itemid["common_itemid_lab_cv_mv"].unique()))
itemid_fill_limit


,51221,50820,50971,50902,50821,50804,50802,50818,50983,50912,...,51407,51366,51045,51495,51368,51017,51062,51377,51391,51122
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
groupby_itemid_lab_MV = mimic_lab_MV.groupby("itemid")

In [8]:
for i, item_id in enumerate(final_common_itemid["common_itemid_lab_cv_mv"]):
    cur_itemid_group = groupby_itemid_lab_MV.get_group(item_id)
    cur_adm_ids_with_cur_itemid = cur_itemid_group["hadm_id"].unique()
    patient_group_for_cur_itemid = cur_itemid_group.groupby("hadm_id")
    cur_itemid_time_until_remeasure_list = []
    for j, hadm_id in enumerate(cur_adm_ids_with_cur_itemid):
        cur_patient_cur_itemid = patient_group_for_cur_itemid.get_group(hadm_id)
        if cur_patient_cur_itemid.shape[0] == 1: # current patient only has one value for cur_itemid
            continue
        else:
            cur_patient_cur_itemid["charttimestamp"] = cur_patient_cur_itemid[["charttime"]].apply((lambda x: datetime.datetime.timestamp(datetime.datetime.strptime(x["charttime"], "%Y-%m-%d %H:%M:%S"))), axis=1)
            cur_patient_cur_itemid.sort_values("charttimestamp", inplace=True)
            cur_patient_cur_itemid.reset_index(drop=True, inplace=True)
            initial_charttime = cur_patient_cur_itemid.iloc[0]["charttimestamp"]
            cur_patient_cur_itemid["interval_from_1st_record"] = ((cur_patient_cur_itemid["charttimestamp"]-initial_charttime) / 3600).astype(int)
            cur_patient_cur_itemid["interval_until_next_remeasure"] = cur_patient_cur_itemid["interval_from_1st_record"].diff()
            cur_itemid_time_until_remeasure_list += list(cur_patient_cur_itemid["interval_until_next_remeasure"].values[1:])
#             print(cur_itemid_time_until_remeasure_list)
#             display(cur_patient_cur_itemid)
    if len(cur_itemid_time_until_remeasure_list) == 0: # no patients record this itemid for multiple times
        itemid_fill_limit.loc[0][item_id] = 0
    else:
        itemid_fill_limit.loc[0][item_id] = np.quantile(cur_itemid_time_until_remeasure_list, 0.8)
    

In [9]:
fill_limit_each_itemid = itemid_fill_limit.T
fill_limit_each_itemid.reset_index(inplace=True)
fill_limit_each_itemid.rename(columns={"index": "itemid", 0: "recommend_fill_limit_MV"}, inplace=True)


In [10]:
fill_limit_each_itemid.columns

Index(['itemid', 'recommend_fill_limit_MV'], dtype='object')

In [11]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(fill_limit_each_itemid)


,itemid,recommend_fill_limit_MV
0,51221,9.0
1,50820,5.0
2,50971,11.0
3,50902,11.0
4,50821,4.0
5,50804,4.0
6,50802,4.0
7,50818,4.0
8,50983,11.0
9,50912,12.0


* **get itemid_fill_limit on mimic_lab_CV data** 

In [14]:
common_itemid_distribution_lab_CV = mimic_lab_CV[mimic_lab_CV["itemid"].isin(common_vars_mv_cv_lab)]["itemid"].value_counts().rename_axis('common_itemid_lab_cv_mv').reset_index(name='counts')
print(common_itemid_distribution_lab_CV.shape)
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(common_itemid_distribution)
final_common_itemid_lab_cv = common_itemid_distribution_lab_CV[common_itemid_distribution_lab_CV["counts"] > 1]
print(final_common_itemid_lab_cv.shape)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(final_common_itemid_lab_cv)


(530, 2)
(507, 2)


,common_itemid_lab_cv_mv,counts
0,50820,89030
1,50800,85901
2,50821,83659
3,50804,83655
4,50818,83654
5,50802,83651
6,51221,68968
7,51265,54283
8,50971,52386
9,50902,50039


In [15]:
final_itemid_num_lab_CV = final_common_itemid_lab_cv.shape[0]

itemid_fill_limit_lab_CV = pd.DataFrame(np.zeros((1, final_itemid_num_lab_CV)), columns=list(final_common_itemid_lab_cv["common_itemid_lab_cv_mv"].unique()))
itemid_fill_limit_lab_CV

,50820,50800,50821,50804,50818,50802,51221,51265,50971,50902,...,51314,51336,51407,51392,51303,51310,51309,51306,51366,51334
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
groupby_itemid_lab_CV = mimic_lab_CV.groupby("itemid")

In [17]:
for i, item_id in enumerate(final_common_itemid_lab_cv["common_itemid_lab_cv_mv"]):
    cur_itemid_group = groupby_itemid_lab_CV.get_group(item_id)
    cur_adm_ids_with_cur_itemid = cur_itemid_group["hadm_id"].unique()
    patient_group_for_cur_itemid = cur_itemid_group.groupby("hadm_id")
    cur_itemid_time_until_remeasure_list = []
    for j, hadm_id in enumerate(cur_adm_ids_with_cur_itemid):
        cur_patient_cur_itemid = patient_group_for_cur_itemid.get_group(hadm_id)
        if cur_patient_cur_itemid.shape[0] == 1: # current patient only has one value for cur_itemid
            continue
        else:
            cur_patient_cur_itemid["charttimestamp"] = cur_patient_cur_itemid[["charttime"]].apply((lambda x: datetime.datetime.timestamp(datetime.datetime.strptime(x["charttime"], "%Y-%m-%d %H:%M:%S"))), axis=1)
            cur_patient_cur_itemid.sort_values("charttimestamp", inplace=True)
            cur_patient_cur_itemid.reset_index(drop=True, inplace=True)
            initial_charttime = cur_patient_cur_itemid.iloc[0]["charttimestamp"]
            cur_patient_cur_itemid["interval_from_1st_record"] = ((cur_patient_cur_itemid["charttimestamp"]-initial_charttime) / 3600).astype(int)
            cur_patient_cur_itemid["interval_until_next_remeasure"] = cur_patient_cur_itemid["interval_from_1st_record"].diff()
            cur_itemid_time_until_remeasure_list += list(cur_patient_cur_itemid["interval_until_next_remeasure"].values[1:])
#             print(cur_itemid_time_until_remeasure_list)
#             display(cur_patient_cur_itemid)
    if len(cur_itemid_time_until_remeasure_list) == 0: # no patients record this itemid for multiple times
        itemid_fill_limit_lab_CV.loc[0][item_id] = 0
    else:
        itemid_fill_limit_lab_CV.loc[0][item_id] = np.quantile(cur_itemid_time_until_remeasure_list, 0.8)
    

In [18]:
fill_limit_each_itemid_lab_CV = itemid_fill_limit_lab_CV.T
fill_limit_each_itemid_lab_CV.reset_index(inplace=True)
fill_limit_each_itemid_lab_CV.rename(columns={"index": "itemid", 0: "recommend_fill_limit_CV"}, inplace=True)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(fill_limit_each_itemid_lab_CV)


,itemid,recommend_fill_limit_CV
0,50820,4.0
1,50800,4.0
2,50821,4.0
3,50804,4.0
4,50818,4.0
5,50802,4.0
6,51221,9.0
7,51265,12.0
8,50971,11.0
9,50902,12.0


* **"fill_limit_each_itemid" : recommend fill limit(quantile 0.8) for mimic_lab_MV**
* **"fill_limit_each_itemid_lab_CV": recommend fill limit(quantile 0.8) for mimic_lab_CV**
* **Only keep those itemid: (1) existed both in "fill_limit_each_itemid" & "fill_limit_each_itemid_lab_CV" (2) fill_limit > 0**

In [19]:
useful_itemid_lab_MV = set(fill_limit_each_itemid[fill_limit_each_itemid["recommend_fill_limit_MV"]>0]["itemid"])
useful_itemid_lab_CV = set(fill_limit_each_itemid_lab_CV[fill_limit_each_itemid_lab_CV["recommend_fill_limit_CV"]>0]["itemid"])
final_common_useful_itemid = useful_itemid_lab_MV.intersection(useful_itemid_lab_CV)
len(final_common_useful_itemid)

239

In [20]:
fill_limit_lab_MV = fill_limit_each_itemid[fill_limit_each_itemid["itemid"].isin(final_common_useful_itemid)]
fill_limit_lab_CV = fill_limit_each_itemid_lab_CV[fill_limit_each_itemid_lab_CV["itemid"].isin(final_common_useful_itemid)]
fill_limit_lab_MV.reset_index(drop=True, inplace=True)
fill_limit_lab_CV.reset_index(drop=True, inplace=True)
print(fill_limit_lab_MV.shape)
print(fill_limit_lab_CV.shape)

(239, 2)
(239, 2)


In [21]:
fill_limit_lab_CV.head()

,itemid,recommend_fill_limit_CV
0,50820,4.0
1,50800,4.0
2,50821,4.0
3,50804,4.0
4,50818,4.0


In [22]:
fill_limit_lab_MV.head()

,itemid,recommend_fill_limit_MV
0,51221,9.0
1,50820,5.0
2,50971,11.0
3,50902,11.0
4,50821,4.0


In [23]:
lab_itemid_fill_limit = fill_limit_lab_CV.merge(fill_limit_lab_MV ,on="itemid")
lab_itemid_fill_limit.reset_index(drop=True, inplace=True)
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(lab_itemid_fill_limit)
display(lab_itemid_fill_limit)

,itemid,recommend_fill_limit_CV,recommend_fill_limit_MV
0,50820,4.0,5.0
1,50800,4.0,5.0
2,50821,4.0,4.0
3,50804,4.0,4.0
4,50818,4.0,4.0
...,...,...,...
234,51206,16.2,13.4
235,51088,14.0,5.6
236,50838,11.4,8.0
237,51273,14.0,3.0


In [24]:
lab_itemid_fill_limit.to_csv("./data/fill_limit_lab.csv", index=False)

In [84]:
fill_limit_each_itemid.head()

,itemid,recommend_fill_limit_MV
0,51221,9.0
1,50820,5.0
2,50971,11.0
3,50902,11.0
4,50821,4.0


In [85]:
print(fill_limit_each_itemid.shape)
print(fill_limit_each_itemid_lab_CV.shape)

(500, 2)
(507, 2)


In [86]:
fill_limit_each_itemid_lab_CV.head()

,itemid,recommend_fill_limit_CV
0,50820,4.0
1,50800,4.0
2,50821,4.0
3,50804,4.0
4,50818,4.0


## itemid in set "regard_as_static_common" will be regarded as static variables for lab data

In [87]:
regard_as_static_MV = fill_limit_each_itemid[fill_limit_each_itemid["recommend_fill_limit_MV"] == 0]
regard_as_static_CV = fill_limit_each_itemid_lab_CV[fill_limit_each_itemid_lab_CV["recommend_fill_limit_CV"] == 0]
regard_as_static_common = set(regard_as_static_MV["itemid"]).intersection(set(regard_as_static_CV["itemid"]))
regard_as_static_common

{50835,
 50841,
 50842,
 50850,
 50877,
 50891,
 50892,
 50913,
 50918,
 50926,
 50932,
 50958,
 50968,
 50969,
 50973,
 50980,
 50988,
 50990,
 50996,
 51015,
 51017,
 51025,
 51034,
 51035,
 51041,
 51042,
 51043,
 51045,
 51049,
 51051,
 51052,
 51060,
 51063,
 51067,
 51068,
 51072,
 51073,
 51084,
 51086,
 51094,
 51108,
 51110,
 51111,
 51122,
 51124,
 51140,
 51152,
 51154,
 51155,
 51156,
 51158,
 51159,
 51160,
 51164,
 51167,
 51168,
 51172,
 51177,
 51178,
 51182,
 51183,
 51184,
 51188,
 51191,
 51192,
 51193,
 51199,
 51203,
 51208,
 51215,
 51217,
 51224,
 51225,
 51226,
 51227,
 51228,
 51230,
 51234,
 51235,
 51238,
 51239,
 51243,
 51262,
 51264,
 51270,
 51272,
 51276,
 51289,
 51303,
 51306,
 51307,
 51309,
 51310,
 51313,
 51314,
 51315,
 51319,
 51320,
 51321,
 51323,
 51324,
 51325,
 51326,
 51328,
 51331,
 51332,
 51333,
 51334,
 51336,
 51337,
 51338,
 51340,
 51341,
 51343,
 51344,
 51345,
 51347,
 51352,
 51357,
 51358,
 51366,
 51369,
 51376,
 51377,
 51383,
